In [87]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

## Helper functions

In [88]:
secrets_resolver = SecretsResolverDemo()

In [89]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [90]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


## Receiving the OOB message

In [91]:
#oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/oob_url").text

In [92]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '1dbb0e86-c0ec-45bc-9c53-711a958cbc77', 'from': 'did:peer:2.Ez6LSc7Abj5qZiNAqrmzNRnzxQEkZZbfNUPPNmwrCw4Gh3zw5.Vz6MknAzBHpSHXsBycQd3mNGphjWiZPt6tJbkDrkLYqzh5d5M.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}


## Creating DID and Message

In [93]:
bob_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")
print("Bob's DID:", bob_did)

Bob's DID: did:peer:2.Ez6LSeX5VyqDWZ4YixckA8TDD63RdAYYkeP6P3PpYFAbbM4TD.Vz6MkfLpj7293CgRaEpPEWAKDR2DRDF74xHXcEEJrdCD7uF9j.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0


Select one message type from action-menu: menu-request or perform

In [94]:
bob_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/action-menu/2.0/menu-request",
    frm = bob_did,
    to = [received_msg_decoded["from"]],
    body = {}
)
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/action-menu/2.0/perform",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#         "name": "form-example1",
#         "params": {
#             "name": "Rodolfo",
#             "lastname": "Miranda"
#         }
#     }
# )
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/action-menu/2.0/perform",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#         "name": "ask-me-example",
#         "params": {
#              "question": "What is the capital of Netherlands",
#          }
#     }
# )

In [95]:
bob_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_response_message,
    frm = bob_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Agent

In [96]:
agent_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
agent_endpoint = agent_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(agent_endpoint, headers=headers, data = bob_packed_msg.packed_msg)

## Decode response from Agent

In [97]:
bob_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(bob_unpack_msg.message.body)

{'title': 'RootsID Issuer', 'description': 'RootsID demo issuer', 'errormsg': None, 'options': [{'name': 'form-example', 'title': 'Form Example', 'description': 'Form to be filled in and submitted', 'form': {'description': 'Please provide the following information', 'params': [{'name': 'name', 'title': 'Name', 'default': '', 'description': '', 'required': True, 'type': 'text'}, {'name': 'lastname', 'title': 'Last name', 'default': '', 'description': '', 'required': True, 'type': 'text'}], 'submit-label': 'Submit form'}}, {'name': 'ask-me-example', 'title': 'Askme anything', 'description': 'Wolfram alpha will respond', 'form': {'description': 'Ask me a question', 'params': [{'name': 'question', 'title': 'Question', 'type': 'text'}], 'submit-label': 'Ask'}}, {'name': 'trigger-questionanswer', 'title': 'Trigger a question ', 'description': 'Trigger a question message using questionanswer protocol'}]}
